In [1]:
%pip install mysql-connector-python

In [2]:
import mysql.connector
import os
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [7]:
db_user = os.environ.get('my_username')
db_password = os.environ.get('my_password')
db = mysql.connector.connect(host= "localhost", user = db_user, passwd = db_password, database = "danny_diner")
db

# Queries

Breaking down how to answer all these questions below in order for danny to decide whether should he expand his business:

1. What is the total amount each customer spent at the restaurant?
2. How many days has each customer visited the restaurant?
3. What was the first item from the menu purchased by each customer?
4. What is the most purchased item on the menu and how many times was it purchased by all customers?
5. Which item was the most popular for each customer?
6. Which item was purchased first by the customer after they became a member?
7. Which item was purchased just before the customer became a member?
8. What is the total items and amount spent for each member before they became a member?
9. If each $1 spent equates to 10 points and sushi has a 2x points multiplier - how many points would each customer have?
10. In the first week after a customer joins the program (including their join date) they earn 2x points on all items, not just sushi - how many points do customer A and B have at the end of January?


Solution Q1

In [9]:
q1 = pd.read_sql_query("select s.customer_id, sum(m.price) as total_price from sales s inner join menu m on s.product_id = m.product_id group by s.customer_id",db)
q1

,customer_id,total_price
0,A,76.0
1,B,74.0
2,C,36.0


Solution Q2

In [10]:
q2 = pd.read_sql_query("select customer_id, count(distinct order_date) from sales group by customer_id",db)
q2

,customer_id,count(distinct order_date)
0,A,4
1,B,6
2,C,2


Solution Q3

In [11]:
q3 = pd.read_sql_query("""with row_num_table as 
(select s.customer_id, m.product_name , row_number() over(partition by s.customer_id order by m.product_name desc) as row_num_each_cust 
from sales s inner join menu m on s.product_id = m.product_id ) 
select customer_id, product_name from row_num_table rnt where rnt.row_num_each_cust = 1""",db)
q3

,customer_id,product_name
0,A,sushi
1,B,sushi
2,C,ramen


Solution Q4

In [12]:
q4 = pd.read_sql_query("select m.product_name, count(s.product_id) as freq from sales s inner join menu m on s.product_id = m.product_id group by s.product_id ORDER BY freq desc limit 1",db)
q4

,product_name,freq
0,ramen,8


Solution Q5

In [13]:
q5 = pd.read_sql_query("""with new_ranking_table as 
(select s.customer_id, m.product_name, count(m.product_id) as freqcount, rank() over(partition by s.customer_id order by count(s.customer_id) ) as newranking  
from sales s inner join menu m on s.product_id = m.product_id GROUP BY s.customer_id, m.product_name)
select customer_id, freqcount, product_name, new_ranking_table.newranking from new_ranking_table where new_ranking_table.newranking = 1""",db)
q5

,customer_id,freqcount,product_name,newranking
0,A,1,sushi,1
1,B,2,curry,1
2,B,2,sushi,1
3,B,2,ramen,1
4,C,3,ramen,1


Solution Q6

In [14]:
q6 = pd.read_sql_query("""with new_ranking as (select s.customer_id, s.order_date, mn.product_id, mn.product_name, dense_rank() over(partition by s.customer_id order by s.order_date asc) as ranking
from sales s 
join members mb on s.customer_id = mb.customer_id
join menu mn on s.product_id = mn.product_id
where s.order_date >= mb.join_date)

select customer_id, order_date, product_name from new_ranking where ranking =1""",db)
q6

,customer_id,order_date,product_name
0,A,2021-01-07,curry
1,B,2021-01-11,sushi


Solution Q7

In [15]:
q7 = pd.read_sql_query("""with new_ranking as (select s.customer_id, s.order_date , mn.product_id, mn.product_name, dense_rank() over(partition by s.customer_id order by s.order_date desc) as ranking
from sales s 
join members mb on s.customer_id = mb.customer_id
join menu mn on s.product_id = mn.product_id
where s.order_date < mb.join_date
order by s.customer_id asc, s.order_date desc)
select customer_id,order_date, product_name from new_ranking where ranking = 1""",db)
q7

,customer_id,order_date,product_name
0,A,2021-01-01,sushi
1,A,2021-01-01,curry
2,B,2021-01-04,sushi


Solution Q8

In [16]:
q8 = pd.read_sql_query("""select s.customer_id, count(distinct mn.product_name), sum(mn.price)
from sales s 
join members mb on s.customer_id = mb.customer_id
join menu mn on s.product_id = mn.product_id
where s.order_date < mb.join_date
group by s.customer_id
order by s.customer_id asc, s.order_date desc""",db)
q8

,customer_id,count(distinct mn.product_name),sum(mn.price)
0,A,2,25.0
1,B,2,40.0


Solution Q9

In [17]:
q9 = pd.read_sql_query("""with point_table as (select s.customer_id, mn.product_name, mn.price, case when mn.product_id = 1 then mn.price*2 else mn.price end as points
from sales s 
left join members mb on s.customer_id = mb.customer_id
join menu mn on s.product_id = mn.product_id
order by s.customer_id)
select customer_id, sum(points*10) as real_points_collected from point_table group by customer_id""",db)
q9

,customer_id,real_points_collected
0,A,860.0
1,B,940.0
2,C,360.0


Solution Q10

In [18]:
q10 = pd.read_sql_query("""select s.customer_id, s.order_date, mb.join_date, mn.product_name,s.product_id, mn.price, case when s.order_date >= mb.join_date then price*2 else price end as ref
from sales s 
join menu mn on s.product_id = mn.product_id
join members mb on s.customer_id = mb.customer_id
order by s.customer_id asc, s.order_date asc""",db)
q10

,customer_id,order_date,join_date,product_name,product_id,price,ref
0,A,2021-01-01,2021-01-07,sushi,1,10,10
1,A,2021-01-01,2021-01-07,curry,2,15,15
2,A,2021-01-07,2021-01-07,curry,2,15,30
3,A,2021-01-10,2021-01-07,ramen,3,12,24
4,A,2021-01-11,2021-01-07,ramen,3,12,24
5,A,2021-01-11,2021-01-07,ramen,3,12,24
6,B,2021-01-01,2021-01-09,curry,2,15,15
7,B,2021-01-02,2021-01-09,curry,2,15,15
8,B,2021-01-04,2021-01-09,sushi,1,10,10
9,B,2021-01-11,2021-01-09,sushi,1,10,20
